In [28]:
from adc import align, count

import urllib3
import shutil
import os

import matplotlib.pyplot as plt
import yaml

import tifffile as tf
from scipy.ndimage import gaussian_filter

In [6]:

def load_data(url, filename):

    if not os.path.exists(filename):
        print(f'loading {filename}')
        c = urllib3.PoolManager()

        with c.request('GET',url, preload_content=False) as resp, open(filename, 'wb') as out_file:
            shutil.copyfileobj(resp, out_file)

        resp.release_conn()
    else:
        print(f'{filename} already exists')
    print(f'reading from disk {filename}')
    return tf.imread(filename)



In [21]:
with open('test_data.yaml','r') as f:
    config  = yaml.load(f, Loader=yaml.SafeLoader)
    print (config)

{'bf_fluo_data': {'filename': '00ng_BF_TRITC_bin2.tif', 'url': 'https://github.com/BaroudLab/anchor-droplet-chip/releases/download/v0.0.1/00ng_BF_TRITC_bin2.tif'}, 'mask2': {'filename': 'labels_bin2.tif', 'url': 'https://github.com/BaroudLab/anchor-droplet-chip/releases/download/v0.0.1/labels_bin2+100.tif'}, 'template16': {'filename': 'template_bin16_bf.tif', 'url': 'https://github.com/BaroudLab/anchor-droplet-chip/releases/download/v0.0.1/template_bin16_bf.tif'}, 'path_to_save': '00ng_aligned.tif', 'binnings': [2, 16, 2], 'constraints': {'scale': [1, 0.2], 'tx': [0, 500], 'ty': [0, 500], 'angle': [0, 30]}}


In [ ]:
align.align_stack()

In [22]:
def prepare_data(key, config=config):
    if 'url' in config[key]:
        return load_data(**config[key])
    else:
        return config[key]


In [23]:
data = {key: prepare_data(key) for key in config}

dataset already exists
dataset already exists
dataset already exists


In [29]:
data['template16'] = gaussian_filter(data['template16'], 2)

In [31]:
data

{'bf_fluo_data': array([[[13152, 13037, 13763, ..., 15475, 15343, 15634],
         [13419, 13460, 13550, ..., 15709, 15629, 15541],
         [12898, 13329, 13564, ..., 15487, 15213, 15218],
         ...,
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  411,   412,   415, ...,   418,   425,   411],
         [  409,   425,   404, ...,   415,   412,   418],
         [  411,   409,   428, ...,   420,   433,   428],
         ...,
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]]], dtype=uint16),
 'mask2': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 'te

In [38]:
aligned_stack, tvec = align.align_stack(**data)

2022-06-23 15:38:24,667 INFO : Aligned stack will be saved to 00ng_aligned.tif


transform (7766, 22244)
transform (7766, 22244)
Saved aligned stack 00ng_aligned.tif


In [39]:
tvec

{'tvec': array([3.07415370e-02, 1.86458206e+03]),
 'success': 0.03186239709182759,
 'angle': -1.8212399539050352,
 'scale': 0.9967318370772736,
 'Dscale': 0.00048305064097878487,
 'Dangle': 0.013485166317051244,
 'Dt': 0.25,
 'timg': None}

In [32]:
import json

In [34]:
tvec['tvec'] = list(tvec['tvec'])

In [36]:
json.dumps([tvec,tvec])

'[{"tvec": [0.030741536959794757, 1864.5820629757582], "success": 0.03186239709182759, "angle": -1.8212399539050352, "scale": 0.9967318370772736, "Dscale": 0.00048305064097878487, "Dangle": 0.013485166317051244, "Dt": 0.25, "timg": null}, {"tvec": [0.030741536959794757, 1864.5820629757582], "success": 0.03186239709182759, "angle": -1.8212399539050352, "scale": 0.9967318370772736, "Dscale": 0.00048305064097878487, "Dangle": 0.013485166317051244, "Dt": 0.25, "timg": null}]'

In [ ]:
def prep_tvec(transform_dict):
    t = transform_dict.copy()
    t["tvec"] = list(t["tvec"])
    return t

def save_tvecs(tvecs, path):
    try:
        transform_data = [map(prep_tvec, tvecs)]
        with open(path, 'w') as f:
            json.dump(transform_data, fp=f)
    except Exception as e:
        print('saving transform json failed: ', e.args)

save_tvecs([tvec,tvec], "test.json")